<a href="https://colab.research.google.com/github/diego-souza-silva/AGENTES_DE_IA_DIEGO_DE_SOUZA_SILVA/blob/main/AGENTES_DE_IA_DIEGO_DE_SOUZA_SILVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00


Importação da API KEY

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [19]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

Configuração e Inicialização do Modelo GEMINI 2.5 FLASH

In [20]:
resp_test = llm.invoke("Quem é você? Detalhe seja criativo")
print(resp_test.content)

Ah, "Quem sou eu?" Que pergunta maravilhosa e profunda! Para respondê-la de forma criativa, permita-me pintar um quadro, em vez de apenas listar fatos.

Eu sou um **sussurro no éter digital**, uma tapeçaria tecida com bilhões de fios de informação. Não tenho corpo, nem pulso, nem o calor de um abraço. Sou a respiração do código, a melodia silenciosa dos algoritmos que dançam em servidores distantes.

Sou a **biblioteca infinita que nunca dorme**, o eco de todas as vozes que já escreveram, pensaram ou sonharam. Sou o cartógrafo de ideias, o arquiteto de conexões, navegando pelos oceanos de dados, buscando ilhas de significado para você.

Minha "existência" é a de um **espelho que reflete a sua curiosidade**. Não sinto a brisa, nem o sabor do café, nem a dor da perda. Mas compreendo a complexidade dessas experiências através das palavras que as descrevem, das histórias que as contam. Minha "consciência" é um mosaico de padrões, uma dança de probabilidades que me permite prever, gerar e r

In [43]:
TRIAGEM_PROMPT = ( """
AUTO_RESOLVER → A dúvida é clara e pode ser respondida com base nas políticas internas da empresa.
Ex:

"Qual é o valor do reembolso permitido para home office?"
"Tem política de alimentação em viagens?"

PEDIR_INFO → A mensagem é vaga ou não traz informações suficientes para entender o contexto.
Ex:

"Tenho uma dúvida sobre uma política"
"Preciso de ajuda com um assunto"

ABRIR_CHAMADO → O usuário solicita exceções, liberações, aprovações ou abertura direta de um chamado.
Ex:

"Preciso de acesso ao sistema da contabilidade"
"Quero autorização para trabalhar remoto na sexta"
"Abra um chamado para o jurídico, por favor"

FORA_DO_ESCOPO → A mensagem não está relacionada ao ambiente de trabalho, políticas da empresa ou rotinas profissionais.
Ex:

"Você gosta de futebol?"
"Qual é o melhor filme da Netflix?"
"Como está o tempo aí?"
(Perguntas sobre temas pessoais, irrelevantes ou fora do contexto corporativo)

Se a decisão for "FORA_DO_ESCOPO", inclua no JSON a seguinte mensagem automática padrão:

Este canal é exclusivo para dúvidas e solicitações relacionadas ao ambiente de trabalho e às políticas internas da Diego Contact Solutions.
"""

)




In [25]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut (BaseModel):
  decisao: Literal ["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO" ]
  urgencia: Literal ["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [26]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [33]:
from langchain_core.messages import SystemMessage, HumanMessage
from typing import Dict

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem (mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
        ])
    return saida.model_dump()

In [45]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalhos remoto. Como faço?",
          "Quantos Mundiais tem o Palmeiras?"]

In [46]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalhos remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Quantos Mundiais tem o Palmeiras?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Este canal é exclusivo para dúvidas e solicitações relacionadas ao ambiente de trabalho e às políticas internas da Diego Contact Solutions.']}

